In [1]:
#pip install --upgrade pip


In [2]:
#pip install html5lib

In [3]:
#pip install selenium

In [4]:
#pip install bs4

In [5]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By



In [6]:
# establish selenium driver to click through the 'load more' buttom 
driver = webdriver.Chrome()
driver.get("https://www.foxnews.com/category/shows/tucker-carlson-tonight/transcript")
button = driver.find_element(By.XPATH, "//div[@class='button load-more js-load-more']") 

for x in range(50):
    button.click()    
    #time.sleep(1)
    
# save the source to feed to beautiful soup 
page_source = driver.page_source 


In [7]:
soup = BeautifulSoup(page_source)
url_soup = soup.find_all('a', href=re.compile('/opinion/.*'))

In [8]:
#url_soup

In [9]:
# create a list of urls for the transcript webpages 

url_list = []
prefix = 'https://www.foxnews.com/'
for link in url_soup: 
    suffix = re.findall('\"\/(.*?)\"', str(link))
    if suffix:
        url_list.append(prefix + suffix[0])
    

In [10]:
# use BeautifulSoup to webscrape each Tucker Carlson webpage and store the output in dfs

full_df = None
dfs = []

for n, url in enumerate(url_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    time = soup.find('time').text
    title = soup.title.string

    a_tag = soup.find_all('a')
    for tag in a_tag:
        tag.decompose()

    copy_tag = soup.find_all('p', class_ = "copyright")
    for tag in copy_tag:
        tag.decompose()

    span_tag = soup.find_all('span')
    for tag in span_tag:
        tag.decompose()

    bold_tag = soup.find_all('strong') 
    for tag in bold_tag:
        tag.decompose()

    italic_tag=soup.find_all('i') 
    for tag in italic_tag:
        tag.decompose()

    other_tag = soup.find_all('p', class_ ="dek")
    for tag in other_tag:
        tag.decompose()

    other_tag2 = soup.find_all('p', class_ ="subscribed hide")
    for tag in other_tag2:
        tag.decompose()

    other_tag3 = soup.find_all('p', class_ ="success hide") 
    for tag in other_tag3:
        tag.decompose()


    final = soup.find_all('p')
    # text=''
    # print(final)
    finaltext = ''
    for p in final:

        try:
            finaltext = finaltext + p.text
        except:
            continue
    
    data={'url': [url],'timestamp': [time], 'title': [title], 'transcript': [finaltext]}
    df = pd.DataFrame(data)
    dfs.append(df)


In [11]:
full_df = pd.concat(dfs).reset_index(drop=True)
#full_df

In [12]:
# store output
full_df.to_csv('Tucker_transcripts.tsv', sep="\t")